In [ ]:
import aiohttp
import asyncio
import bs4
import datetime
from io import BytesIO
import pandas as pd
import PyPDF2
import requests
import time
from tqdm.contrib.telegram import tqdm
import os

In [ ]:
TOKEN = os.getenv('TOKEN')
CHAT_ID = os.getenv('CHAT_ID')

In [20]:
party_news = "https://uzlidep.uz/uz/news-of-party?page={page}" #1250
uzbekistan_news = "https://uzlidep.uz/uz/news-of-uzbekistan?page={page}" #149

uzb_last = 149
party_last = 1250

In [6]:
BASE_URL = "https://uzlidep.uz/"

In [19]:

# scraper.py
import requests
from bs4 import BeautifulSoup

url = 'https://uzlidep.uz/uz/news-of-uzbekistan?page=2'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'lxml')
quotes = soup.find('div', class_='publications list').find_all('a')

for i in quotes:
    print(i['href'])

# print(quotes)
   

/uz/news-of-uzbekistan/12984
/uz/news-of-uzbekistan/12984
/uz/news-of-uzbekistan/12936
/uz/news-of-uzbekistan/12936
/uz/news-of-uzbekistan/12892
/uz/news-of-uzbekistan/12892
/uz/news-of-uzbekistan/12776
/uz/news-of-uzbekistan/12776
/uz/news-of-uzbekistan/12718
/uz/news-of-uzbekistan/12718
/uz/news-of-uzbekistan/12684
/uz/news-of-uzbekistan/12684
/uz/news-of-uzbekistan/12665
/uz/news-of-uzbekistan/12665
/uz/news-of-uzbekistan/12641
/uz/news-of-uzbekistan/12641
/uz/news-of-uzbekistan/12462
/uz/news-of-uzbekistan/12462
/uz/news-of-uzbekistan/12436
/uz/news-of-uzbekistan/12436


In [21]:
links = []
async def main():
    async with aiohttp.ClientSession() as session:
        for page in tqdm(range(1, uzb_last+1), token=TOKEN, chat_id=CHAT_ID):
            try:
                async with session.get(uzbekistan_news.format(page=page)) as resp:
                    if resp.status == 200:
                        data = await resp.text()
                        soup = bs4.BeautifulSoup(data)
                        selected_links = soup.find('div', class_='publications list').find_all('a')
                        links.extend([BASE_URL+link['href'] for link in selected_links])
                    elif resp.status != 404:
                        await asyncio.sleep(60) # 1 minute
            except Exception as e:
                print(e)
await main()

  0%|          | 0/149 [00:00<?, ?it/s]

In [ ]:
links2 = []
async def main():
    async with aiohttp.ClientSession() as session:
        for page in tqdm(range(1, party_last+1), token=TOKEN, chat_id=CHAT_ID):
            try:
                async with session.get(party_news.format(page=page)) as resp:
                    if resp.status == 200:
                        data = await resp.text()
                        soup = bs4.BeautifulSoup(data)
                        selected_links = soup.find('div', class_='publications list').find_all('a')
                        links2.extend([BASE_URL+link['href'] for link in selected_links])
                    elif resp.status != 404:
                        await asyncio.sleep(60) # 1 minute
            except Exception as e:
                print(e)
await main()

  0%|          | 0/1250 [00:00<?, ?it/s]

In [28]:
results = []
async def main():
    async with aiohttp.ClientSession() as session:
        for link in tqdm(links, token=TOKEN, chat_id=CHAT_ID):
            try:
                async with session.get(link) as resp:
                    if resp.status == 200:
                        data = await resp.text()
                        soup = bs4.BeautifulSoup(data)
                        content = soup.find('div', class_="full-text").text
                        title = soup.find('h1', class_="page-title").text
                        if len(content) > 100:
                            results.append({
                                "title": title.strip(),
                                "content": content.strip()
                            })
                    elif resp.status != 404:
                        await asyncio.sleep(60) # 1 minute
            except Exception as e:
                print(e)
await main()

  0%|          | 0/2970 [00:00<?, ?it/s]

In [ ]:
results2 = []
async def main():
    async with aiohttp.ClientSession() as session:
        for link in tqdm(links2, token=TOKEN, chat_id=CHAT_ID):
            try:
                async with session.get(link) as resp:
                    if resp.status == 200:
                        data = await resp.text()
                        soup = bs4.BeautifulSoup(data)
                        content = soup.find('div', class_="full-text").text
                        title = soup.find('h1', class_="page-title").text
                        if len(content) > 100:
                            results2.append({
                                "title": title.strip(),
                                "content": content.strip()
                            })
                    elif resp.status != 404:
                        await asyncio.sleep(60) # 1 minute
            except Exception as e:
                print(e)
await main()

/usr/local/lib/python3.8/dist-packages/tqdm/contrib/telegram.py:35: TqdmWarning: Creation rate limit: try increasing `mininterval`.
  self.message_id


  0%|          | 0/24988 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/tqdm/contrib/telegram.py:63: TqdmWarning: Creation rate limit: try increasing `mininterval`.
  message_id = self.message_id


In [38]:
pd.DataFrame(main).to_pickle('MainFiles/uzlidep.pickle')

In [36]:
main = results + results2

In [37]:
len(main)

27838

In [35]:
results2

[{'title': 'Shavkat Mirziyoyev surxondaryolik saylovchilar bilan uchrashdi',
  'content': 'Tadbirkorlar va ishbilarmonlar harakati – O’zbekiston Liberal-demokratik partiyasidan O’zbekiston Respublikasi Prezidentligiga nomzod Shavkat Mirziyoyev 22 iyun kuni Surxondaryo viloyati saylovchilari bilan uchrashdi.\n\nTadbirni nomzodning Surxondaryo viloyati bo‘yicha ishonchli vakili Abdulxakim Eshmuratov ochdi. U o‘z so‘zida Shavkat Mirziyoyev rahbarligida iqtisodiyot jadal rivojlantirilib, ulkan bunyodkorlik ishlari amalga oshirilayotgani, xalqimizga munosib sharoitlar yaratilayotganini ta’kidladi.\n\nJumladan, so‘nggi olti yarim yilda yalpi hududiy mahsulot 1,3 barobarga ko‘payib, 35 trillion so‘mdan oshgan, aholi jon boshiga 13 million so‘mga yetgan. Shu davrda viloyatga jami 5 milliard dollar investitsiya jalb qilingan. Sanoat, qishloq xo‘jaligi, turizm va xizmat ko‘rsatish sohalarida 600 mingdan ortiq yangi ish o‘rni yaratilgan.\n\nIjtimoiy sohada ham ko‘plab ijobiy natijalarga erishilmo